In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "COVID-19"
cohort = "GSE185658"

# Input paths
in_trait_dir = "../../input/GEO/COVID-19"
in_cohort_dir = "../../input/GEO/COVID-19/GSE185658"

# Output paths
out_data_file = "../../output/preprocess/COVID-19/GSE185658.csv"
out_gene_data_file = "../../output/preprocess/COVID-19/gene_data/GSE185658.csv"
out_clinical_data_file = "../../output/preprocess/COVID-19/clinical_data/GSE185658.csv"
json_path = "../../output/preprocess/COVID-19/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19"
!Series_summary	"Balanced immune responses in airways of patients with asthma are crucial to succesful clearance of viral infection and proper asthma control."
!Series_summary	"We used microarrays to detail the global programme of gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo."
!Series_overall_design	"Bronchial brushings from control individuals and patients with asthma around two weeks before (day -14) and four days after (day 4) experimental in vivo rhinovirus infection were used for RNA isolation and hybrydyzation with Affymetric microarrays."
Sample Characteristics Dictionary:
{0: ['time: DAY14', 'time: DAY4'], 1: ['group: AsthmaHDM', 'group: AsthmaHDMNeg', 'group: Healthy'], 2: ['donor: DJ144', 'donor: DJ113', 'donor: DJ139', 'donor

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this is microarray data from bronchial brushings
# which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# After reviewing the data, it's clear this dataset is about asthma and rhinovirus, not COVID-19
# Therefore, the COVID-19 trait we're interested in is not available in this dataset
trait_row = None  # COVID-19 trait information is not available
age_row = None  # Age information is not available
gender_row = None  # Gender information is not available

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait information to binary values for COVID-19"""
    # Since the trait is not available, this function won't be used
    return None

def convert_age(value):
    """Convert age information to continuous values"""
    # Not applicable as age data is not available
    return None

def convert_gender(value):
    """Convert gender information to binary values"""
    # Not applicable as gender data is not available
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None (COVID-19 trait data is not available)


A new JSON file was created at: ../../output/preprocess/COVID-19/cohort_info.json


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/COVID-19/GSE185658/GSE185658_family.soft.gz
Matrix file: ../../input/GEO/COVID-19/GSE185658/GSE185658_series_matrix.txt.gz
Found the matrix table marker at line 63
Gene data shape: (32321, 48)
First 20 gene/probe identifiers:
['7892501', '7892502', '7892503', '7892504', '7892505', '7892506', '7892507', '7892508', '7892509', '7892510', '7892511', '7892512', '7892513', '7892514', '7892515', '7892516', '7892517', '7892518', '7892519', '7892520']


### Step 4: Gene Identifier Review

In [5]:
# These don't appear to be human gene symbols but rather probe identifiers from a microarray platform
# They are numeric identifiers that likely need to be mapped to gene symbols
# Based on my biomedical knowledge, human gene symbols are typically alphanumeric (like BRCA1, TP53, etc.)
# These look like Illumina BeadChip probe IDs which require mapping to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Examine the columns to find gene information
print("\nExamining gene mapping columns:")
print("Column 'ID' examples:")
id_samples = gene_annotation['ID'].head(5).tolist()
for i, sample in enumerate(id_samples):
    print(f"Example {i+1}: {sample}")

# Look at gene_assignment column which contains gene symbols embedded in text
print("\nColumn 'gene_assignment' examples (contains gene symbols):")
if 'gene_assignment' in gene_annotation.columns:
    # Display a few examples of the gene_assignment column
    gene_samples = gene_annotation['gene_assignment'].head(3).tolist()
    for i, sample in enumerate(gene_samples):
        print(f"Example {i+1}: {sample[:200]}...")  # Show first 200 chars
    
    # Extract some gene symbols to verify
    print("\nExtracted gene symbols from gene_assignment:")
    for i, sample in enumerate(gene_samples[:3]):
        symbols = extract_human_gene_symbols(sample)
        print(f"Example {i+1} extracted symbols: {symbols}")
    
    # Identify the columns needed for gene mapping
    print("\nColumns identified for gene mapping:")
    print("- 'ID': Contains probe IDs")
    print("- 'gene_assignment': Contains gene information from which symbols can be extracted")
else:
    print("Error: 'gene_assignment' column not found in annotation data.")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']
{'ID': ['7896736', '7896738', '7896740'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008'], 'seqname': ['chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091'], 'RANGE_STOP': ['54936', '63887', '70008'], 'total_probes': [7.0, 31.0, 24.0], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // --- // --- /// ENST00000492842 // OR4G11P // olfactory receptor, family 4, subfamily G, member 11 pseudogene // --- // --- /// ENST00000588632 // OR4G1P // olfactory receptor, family 4, subf

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze the gene identifiers in the expression data and gene annotation data
# Based on the preview, the 'ID' column in gene_annotation corresponds to the probe IDs in gene_data
# The gene symbols are in the 'gene_assignment' column and need to be extracted

# Define a more specific extraction function for this dataset format
def extract_genes_from_assignment(text):
    """Extract gene symbols from gene_assignment field with specific format handling for this dataset"""
    if not isinstance(text, str) or text == '---':
        return []
    
    genes = []
    # Gene symbols appear after '//' in the format "ID // GENE // description"
    parts = text.split('///')
    for part in parts:
        subparts = part.split('//')
        if len(subparts) > 1 and len(subparts[1].strip()) > 0:
            gene = subparts[1].strip()
            if gene != '---':
                genes.append(gene)
    return genes

# 2. Create the gene mapping dataframe
# We'll use the 'ID' column and extract gene symbols from 'gene_assignment' column
mapping_df = gene_annotation[['ID', 'gene_assignment']].copy()

# Process the mapping dataframe
mapping_df = mapping_df.dropna(subset=['gene_assignment'])  # Drop rows without gene assignments

# Use our custom extraction function instead of the generic one
mapping_df['Gene'] = mapping_df['gene_assignment'].apply(extract_genes_from_assignment)

# Check intermediate results
print("Sample gene mapping (before filtering):")
print(mapping_df[['ID', 'Gene']].head(5))

# Only keep rows that have at least one gene symbol
mapping_df = mapping_df[mapping_df['Gene'].apply(len) > 0]
print(f"Mapping entries with gene symbols: {len(mapping_df)}")

# Make sure IDs are strings
mapping_df['ID'] = mapping_df['ID'].astype(str)

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
expression_df = get_genetic_data(matrix_file)

# Check if our probe IDs match the expression data index
common_ids = set(mapping_df['ID']) & set(expression_df.index.astype(str))
print(f"Number of probe IDs in mapping that match expression data: {len(common_ids)}")

# Create a custom mapping function for debugging
def custom_map_probes_to_genes():
    # Dictionary to store summed expression values for each gene
    gene_expr = {}
    
    # Process each probe
    for idx, row in mapping_df.iterrows():
        probe_id = row['ID']
        genes = row['Gene']
        
        # Skip if probe not in expression data
        if probe_id not in expression_df.index:
            continue
            
        # Skip if no genes to map to
        if len(genes) == 0:
            continue
            
        # Get probe expression values
        probe_values = expression_df.loc[probe_id].to_dict()
        
        # Distribute expression values among genes
        weight = 1.0 / len(genes)
        for gene in genes:
            if gene not in gene_expr:
                gene_expr[gene] = {col: 0 for col in expression_df.columns}
            
            # Add weighted expression to each gene
            for col, val in probe_values.items():
                gene_expr[gene][col] += val * weight
    
    # Convert to dataframe
    result = pd.DataFrame.from_dict(gene_expr, orient='index')
    return result

# Try our custom mapping approach for more control over the process
gene_data = custom_map_probes_to_genes()

# Print information about the mapping and resulting gene expression data
print(f"Original expression data shape: {expression_df.shape}")
print(f"Gene mapping entries: {len(mapping_df)}")
print(f"Resulting gene expression data shape: {gene_data.shape}")
if gene_data.shape[0] > 0:
    print(f"First 10 gene symbols: {gene_data.index[:10].tolist()}")
else:
    print("No gene symbols were mapped successfully.")

# If custom mapping failed, try using the library function with explicit exploded mapping
if gene_data.shape[0] == 0:
    print("Trying alternative mapping approach with exploded mapping dataframe...")
    
    # Create an exploded version with one gene per row
    exploded_df = mapping_df.explode('Gene')
    exploded_df = exploded_df[exploded_df['Gene'] != '']  # Remove empty strings
    
    # Apply the gene mapping using the library function
    gene_data = apply_gene_mapping(expression_df, exploded_df[['ID', 'Gene']])
    
    print(f"Resulting gene expression data shape with exploded mapping: {gene_data.shape}")
    if gene_data.shape[0] > 0:
        print(f"First 10 gene symbols: {gene_data.index[:10].tolist()}")


Sample gene mapping (before filtering):
        ID                                               Gene
0  7896736                                                 []
1  7896738                          [OR4G2P, OR4G11P, OR4G1P]
2  7896740  [OR4F4, OR4F17, OR4F5, OR4F17, OR4F4, OR4F5, O...
3  7896742  [LOC728323, LOC101060626, LOC101060626, LOC101...
4  7896744  [OR4F29, OR4F3, OR4F16, OR4F21, OR4F21, OR4F3,...
Mapping entries with gene symbols: 25293


Number of probe IDs in mapping that match expression data: 24520


Original expression data shape: (32321, 48)
Gene mapping entries: 25293
Resulting gene expression data shape: (25745, 48)
First 10 gene symbols: ['MT-TM', 'FAM87B', 'FAM87A', 'LINC01128', 'SAMD11', 'KLHL17', 'PLEKHN1', 'ISG15', 'AGRN', 'MIR200B']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if clinical data is available before trying to extract features
if trait_row is not None:
    # Extract clinical features only if trait_row exists
    clinical_features = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    
    # Handle missing values and check for bias
    linked_data = handle_missing_values(linked_data, trait)
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
else:
    # No clinical data available
    print("No clinical data available for the trait of interest.")
    linked_data = pd.DataFrame()  # Empty dataframe
    is_biased = True  # Dataset is biased since we have no trait data

# 6. Validate and save cohort info
is_trait_available = trait_row is not None
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data but lacks COVID-19 trait information."
)

# 7. Save the linked data if it's usable
if is_usable:
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save the linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved due to quality issues.")

Gene data shape after normalization: (24776, 48)


Normalized gene data saved to ../../output/preprocess/COVID-19/gene_data/GSE185658.csv
No clinical data available for the trait of interest.
Abnormality detected in the cohort: GSE185658. Preprocessing failed.
Linked data not saved due to quality issues.
